# Binary Search Trees

## Trees

A tree is a graph data structure containing no cycles:
- Each node has zero to many outgoing edges to other nodes (called **children**)
- Each node has at most one incoming edge from another node (called the **parent**). 
- The tree should have only one node with no parent, called the **root**. 
- A graph may be able to be re-arranged into different trees, depending on its structure. 

Trees have a **height**, which is the longest possible distance from the root to a leaf node:
- An empty tree has height -1
- A single node tree has height 0. 

The following trees have height 0, 1, and 2 respectively:
```
   0

   1 
  / \
 3   4 

     0
    / \
   1   2
  /   / 
 3   5   
```

## Binary Trees

**m-ary** trees are trees where no node has more than m children. The most common case of this in CS are **binary trees**, where m=2. The above trees are binary trees (each node has at most 2 children) and both have height 2. A binary tree is **full** or **complete** if all non-leaf nodes have 2 children, and no leaf nodes exist on a higher level than the height of the tree (except if there are insufficent nodes) the leftmost nodes would be filled first):
```
      0
    /   \
   1     2
  / \   / \
 3   4 5   6
```

- A binary tree with height `h` has between (2^h)+1 and 2^(h+1)-1 nodes.

Binary trees are typically be constructed either with custom types for nodes or as an array (though there are many other representations). For example, we can represent the above tree using a custom types:

In [1]:
class TreeNode:
    def __init__(self, val=None, parent=None, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
        self.parent = parent # Some BSTs use nodes without references to parents

        
tree_dict = {val: TreeNode(val) for val in range(7)}
tree_dict[0].left = tree_dict[1]
tree_dict[0].right = tree_dict[2]
tree_dict[1].left = tree_dict[3]
tree_dict[1].right = tree_dict[4]
tree_dict[1].parent = tree_dict[0]
tree_dict[2].left = tree_dict[5]
tree_dict[2].right = tree_dict[6]
tree_dict[2].parent = tree_dict[0]
tree_dict[3].parent = tree_dict[1]
tree_dict[4].parent = tree_dict[1]
tree_dict[5].parent = tree_dict[2]
tree_dict[6].parent = tree_dict[2]
root = tree_dict[0]

Or we can use an array; when representing a tree with an array, the root is put at index 0. The left and right child of a node at index i are 2i+1 and 2i+2. This format can be more compact and easy to work with:

In [2]:
tree_as_arr = [0,1,2,3,4,5,6]

## Binary search trees

A **binary search tree (BST)** is a special case of a binary tree, where for each node the binary search property holds for the node and its children (if any): `left.val <= val <= right.val`. BSTs may or may not contain nodes with duplicate values. These are binary search trees:
```
   1      2       3
  / \      \
 0   4      5
```
and these are binary trees that are not BSTs:
```
   1      2       
  / \      \
 3   -1      1
```

#### Exercise (CLRS 12.1-2)
What is the difference between the binary-search-tree property and the min-heap property (see page 153)?  Can the min-heap property be used to print out the keys of an n-node tree in sorted order in O(n) time? Show how, or explain why not,

**Solution**

The min-heap property is a looser constraint in that for some node `v` with children `v_l` and `v_r`, `v.value <= v_l.value` and `v.value <= v_r.value`. Because of this, we know that parents and children have an ordered relationship, but left children and right children do not, unlike with a BST. A valid minheap can have multiple possible in-order traversals, whereas  As such, we'd need to do additional work to get a sorted ordering - the typical way with a minheap would be to continuously remove the minimum element. 

## Operations

The most common operations on binary trees and BSTs are:
- Traversal
- Searching (for a value, minimum, maximum, successor, or predecessor)
- Insertion
- Deletion
- Verification

### Traversal

Tree traversal refers to visiting every node in the tree and processing its data. Depending on the goal of the traversal, "processing" can mean almost anything - the most basic example is just printing the value of the node. 

**Running time**: `O(n)` for `n` nodes in the tree; we must visit every node in the tree.

Traversing the tree creates an ordering of its nodes; there are several possible orderings.
- **Pre-order**: Process the current node, visit the left child, visit the right child.
- **In-order**: Visit the left child, process the current node, visit the right child.
- **Post-order**: Visit the left child, visit the right child, process the current node.
- **Level-order**: All nodes at the given level are visited before any of their children.

> The trace of a traversal is called a sequentialisation of the tree. The traversal trace is a list of each visited root. No one sequentialisation according to pre-, in- or post-order describes the underlying tree uniquely. Given a tree with distinct elements, either pre-order or post-order paired with in-order is sufficient to describe the tree uniquely. ([Wikipedia](https://en.wikipedia.org/wiki/Tree_traversal#In-order_(LNR)))

Note that pre/in/post order traversals can be used to implement depth first searches, while level-order traversals can be used to implement breadth first searches. Orderings may be reversed by visiting the right child before the left. Depth-first traversals can be implemented iteratively using a stack or recursively (implicitly using the call stack):

In [3]:
from typing import List
"""
      0
    /    \
   1      2
  / \    / \ 
 3   4  5   6 
"""

def pre_order_recursive(node: TreeNode):
    if not node:
        return
    print(node.val, end =" ")
    pre_order_recursive(node.left) if node.left else None
    pre_order_recursive(node.right) if node.right else None
pre_order_recursive(root)

print("\n"+("-"*14))

def pre_order_recursive(tree: List[int], i: int):
    if not (0 <= i <=len(tree)-1):
        return
    print(tree[i], end =" ")
    pre_order_recursive(tree, 2*i+1) if 2*i+1 < len(tree) else None
    pre_order_recursive(tree, 2*i+2) if 2*i+2 < len(tree)  else None
pre_order_recursive(tree_as_arr, 0)

0 1 3 4 2 5 6 
--------------
0 1 3 4 2 5 6 

In [4]:
def pre_order_iterative(node: TreeNode):
    stack = [node]
    while stack:
        curr = stack.pop()
        print(curr.val, end =" ")
        # Note that right is pushed first when using a stack explicitly,
        # causing it to be evaluated after left.
        stack.append(curr.right) if curr.right else None 
        stack.append(curr.left) if curr.left else None
pre_order_iterative(root)

print("\n"+("-"*14))

def pre_order_iterative(tree: List[int]):
    if not tree:
        return
    stack = [0]
    while stack:
        i = stack.pop()
        print(i, end =" ")
        stack.append(2*i+2) if 2*i+2 < len(tree) else None 
        stack.append(2*i+1) if 2*i+1 < len(tree)  else None
pre_order_iterative(tree_as_arr)

0 1 3 4 2 5 6 
--------------
0 1 3 4 2 5 6 

Note that in the above, we can easily change from pre- to in- or post- order by simply reordering the calls to 
`print()` and visitng the children.

In [5]:
def in_order_recursive(node: TreeNode):
    if not node:
        return
    in_order_recursive(node.left) if node.left else None
    print(node.val, end =" ")
    in_order_recursive(node.right) if node.right else None
in_order_recursive(root)

3 1 4 0 5 2 6 

Level order traversal is implemented iteratively, but using a queue (most easily via `collections.deque()` in Python):

In [6]:
from collections import deque

def level_order_iterative(node: TreeNode):
    q = deque([node])
    while q:
        curr = q.popleft()
        print(curr.val, end =" ")
        q.append(curr.left) if curr.left else None
        q.append(curr.right) if curr.right else None 
level_order_iterative(root)

print("\n"+("-"*14))

def level_order_iterative(tree: List[int]):
    if not tree:
        return
    q = deque([0])
    while q:
        i = q.popleft()
        print(i, end =" ")
        q.append(2*i+1) if 2*i+1 < len(tree) else None
        q.append(2*i+2) if 2*i+2 < len(tree) else None 
level_order_iterative(tree_as_arr)

0 1 2 3 4 5 6 
--------------
0 1 2 3 4 5 6 

#### Exercise (CLRS 12.1-3)
Give a nonrecursive algorithm that performs an inorder tree walk. (Hint: An easy solution uses a stack as an auxiliary data structure.  A more complicated, but elegant, solution uses no stack but assumes that we can test two pointers for equality.)

#### Solution
The stack-based approach is shown above. For the two-pointer solution, CLRS makes a critical distinction in its definition of BSTs:
> We can represent such a tree by a linked data structure in which each node is an object.  In addition to a key and satellite data, each node contains attributes `left`, `right`, and `p` that point to the nodes corresponding to its left child, its right child, and its parent, respectively. (p. 286-287)

With a pointer to the parent, we can leverage the fact that the root of the tree is the only node without a parent, and use the following approach:
```
Start at the root node, and do the following until our current node is nil:
    - If there's a left and we just came from the parent, go left.
    - If there's no left or we just came from the left:
        - print this node's value.
        - If there is a right, go right
    - If we just came from the right or there is no right:
    - Otherwise Go to the parent
 ```

In [16]:
"""
      0
    /    \
   1      2
  / \    / \ 
 3   4  5   6 
"""

def inorder_traversal(root):
    current = root
    past = None
    while current:
        # Go left if we just came from the parent and a left child exists.
        if (past == current.parent) and current.left:
            past = current
            current = current.left
            continue
        # Print the current node's value if we just came from the left or can't go left.
        if (past == current.left) or (not current.left):
            print(current.val, end=" ")
        # Go right if we just came from the left and can go right.
        if (past == current.left) and current.right:
            past = current
            current = current.right
            continue
        # Otherwise return to the parent 
        past = current
        current = current.parent
inorder_traversal(root)

3 1 4 0 5 2 6 

### Searching

#### Running Time
`O(log(n))` for a BST of n nodes (`O(n)` for a binary tree). A BST search exploits the binary search property and excludes half of a node's subtree at each step, whereas a regular binary tree has no ordering so a regular traversal must be used. As with traversals, searches can be implemented recursively or iteratively, using a node class or array.

#### key
Find a specific key in the tree if it exists.

In [17]:
"""
      5
    /   \
   3     7 
  / \   / \
 1   4 6   8
"""

bst_dict = {val: TreeNode(val) for val in range(1,9)}
bst_dict[5].left = bst_dict[3]
bst_dict[5].right = bst_dict[7]
bst_dict[3].left = bst_dict[1]
bst_dict[3].right = bst_dict[4]
bst_dict[7].left = bst_dict[6]
bst_dict[7].right = bst_dict[8]
bst_root = bst_dict[5]

def bst_search(node: TreeNode, key: int):
    if not node:
        return False
    if node.val == key:
        return True
    if node.val > key:
        return bst_search(node.left, key)
    else:
        return bst_search(node.right, key)

assert bst_search(bst_root, 3) 
assert not bst_search(bst_root, 42)

bst_arr = [5,3,7,1,4,6,8]

def bst_search(tree: List[int], i: int, key: int):
    if not tree or i >= len(tree):
        return False
    if tree[i] == key:
        return True
    if tree[i] > key:
        return bst_search(tree, 2*i+1, key)
    else:
        return bst_search(tree, 2*i+2, key)

assert bst_search(bst_arr, 0, 3) 
assert not bst_search(bst_arr, 0, 42)

#### min/max
Find the minimum or maximum value in the tree, by following the left or right child pointers from the root, respectively.

In [24]:
def bst_min(node):
    if not node:
        return
    if not node.left:
        return node.val
    return bst_min(node.left)

print(bst_min(bst_root))

def bst_max(node):
    if not node:
        return
    if not node.right:
        return node.val
    return bst_max(node.right)

print(bst_max(bst_root))

1
8


#### predecessor / successor
For a given node `n` in the tree, find elelment `m` in the tree that immediately follows `n` in an in-order traversal (if such element exists).
- If `n` has a right child, `bst_min(n.right)`; the minimal element of the right subtree is the successor.
- If `n` has no right child, search upwards until we reach a node `l` who is the left of child of its parent; `l.parent` is the successor.

### Insertion

- Insertion into a binary tree can be done in O(1) time with an array or log(n) time with TreeNode, since there's no order. 
- BST insertion takes log(n) time so that the ordering can be preserved. 